# Ex) Google PageRank

$$
\newcommand{\eg}{{\it e.g.}}
\newcommand{\ie}{{\it i.e.}}
\newcommand{\argmin}{\operatornamewithlimits{argmin}}
\newcommand{\mc}{\mathcal}
\newcommand{\mb}{\mathbb}
\newcommand{\mf}{\mathbf}
\newcommand{\minimize}{{\text{minimize}}}
\newcommand{\diag}{{\text{diag}}}
\newcommand{\cond}{{\text{cond}}}
\newcommand{\rank}{{\text{rank }}}
\newcommand{\range}{{\mathcal{R}}}
\newcommand{\null}{{\mathcal{N}}}
\newcommand{\tr}{{\text{trace}}}
\newcommand{\dom}{{\text{dom}}}
\newcommand{\dist}{{\text{dist}}}
\newcommand{\R}{\mathbf{R}}
\newcommand{\SM}{\mathbf{S}}
\newcommand{\ball}{\mathcal{B}}
\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}
$$

__<div style="text-align: right"> EE370: Software lab, Kyung Hee University. </div>__
_<div style="text-align: right"> Jong-Han Kim (jonghank@khu.ac.kr) </div>_



The Google PageRank, named after Larry Page, one of the co-founders of Google, is an algorithm that ranks the webpages based on the hyperlink connections of the webpages on relevant topics, hence the Google search can return the list of the webpages in an appropriate order of the PageRank indices.

According to Google, the PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

  <img src="https://upload.wikimedia.org/wikipedia/en/8/8b/PageRanks-Example.jpg" width="500">

The PageRank algorithm outputs a probability distribution used to represent the likelihood that a person randomly clicking on links will arrive at any particular page. 

Suppose there are $n$ webpages on a specific topic, where some of them have hyperlinks to others. Then the PageRank of the webpage $i$, $p_i$, can be represented by

$$
  p_i = \frac{1-\zeta}{n} + \sum_{j\in\mc{N}_i} \zeta\frac{p_j}{d_j}
$$

where 
- $d_j$ is the number of outbound links from the webpage $j$
- $\mc{N}_i$ is the set of webpages that have the outbound links to the webpage $i$
- $\zeta>0$ is the damping factor, which can be interpreted as the probability that a person is not satisfied with the current webpage and clinks on the links to the other pages, and we normally set $\zeta=0.85$


The above relation can be compactly written by the following matrix form,

$$
  p = \frac{1-\zeta}{n} {\bf 1} + \zeta M p
$$

where $p = \bmat{p_1 & p_2 & \cdots & p_n}^T$, and ${\bf 1}\in\R^{n}$ is the one-vector whose elements are all 1's.

1. What is $M_{ij}$, for $i,j \in \{1,\dots,n\}$?



> ---
> `A. M은 i와 j사이 링크가 없다면 0. 존재한다면 j사이트 내 존재하는 링크 중 i사이트를 향하는 링크의 비율을 나타낸다.`
>
> $$
M_{ij} = 
\begin{cases}
  ? &\quad \text{if $j\in\mc{N}_i$ ($j$ has outbound link to $i$)} \\
  ? &\quad \text{otherwise}
\end{cases}
$$
>
> ---


Once you obtained $M\in\R^{n\times n}$, you can solve the above linear equations for $p$. In other words, you can obtain the PageRank for all the $n$ webpages by 

$$
  p = \frac{1-\zeta}{n}\left(I - \zeta M \right)^{-1} {\bf 1}
$$

However in most cases where the number of webpages that contain somethings you are interested in is extremely large, computing the inverse appearing above is practically impossible. For example googling "BTS" returns approximately 528 million documents; you won't be able to compute the inverse of $528000000\times 528000000$ matrix.

Instead of this direct method, you can also solve for $p$ by using the following iterative method. 

$$
  p^{k+1} = \frac{1-\zeta}{n} {\bf 1} + \zeta M p^{k}
$$

where $p^k$ stands for the page rank $p$ at the $k$-th iteration step. From a random initial condition $p^0$, you can keep updating $p^k$ until the update amount is sufficiently small. It turns out that this update rule converges for $0<\zeta<1$.

Now a toy example. Suppose we have the small space of 11 webpages whose list of outbound link connections are given by

- Page A has links to A,B,C,D,E,F,G,H,I,J,K (everyone)
- Page B has links to C
- Page C has links to B
- Page D has links to A, B
- Page E has links to B, D, F
- Page F has links to B, E
- Page G has links to B, E
- Page H has links to B, E
- Page I has links to B, E
- Page J has links to E
- Page K has links to E


2. Build $M$. In other words, define a $11\times 11$ matrix $M$ and assign the explicit numeric value for each $M_{ij}$.

In [3]:
import numpy as np

n = 11
M = np.zeros((n,n)) 
# 각 열(A,B...)에 행으로의 링크가 있다면 1

M[:,0] = 1 # :면 0열 전부 1로 채울 수 있음.
M[2,1] = 1           
M[1,2] = 1           
M[[0,1],3] = 1       
M[[1,3,5],4] = 1    
M[[1,4],5] = 1      
M[[1,4],6] = 1      
M[[1,4],7] = 1       
M[[1,4],8] = 1       
M[4,9] = 1          
M[4,10] = 1         

print (M)

M /= np.sum(M,axis=0) # 행렬 합 실행. axis=0이면 열의 합 진행. 후에 합으로 열을 나눠준다!

print (M)


[[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.09090909 0.         0.         0.5        0.         0.
  0.         0.         0.         0.         0.        ]
 [0.09090909 0.         1.         0.5        0.33333333 0.5
  0.5        0.5        0.5        0.         0.        ]
 [0.09090909 1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.09090909 0.         0.         0.         0.33333333 0.
  0.         0.         0.         0.         0.        ]
 [0.09090909 0.         0.         0.         0.         0.5
  0.5        0.5        0.5        1.         1.        ]
 [0.0909090

3. Randomly generate initial $p^0>0$ such that $\|p^0\|_1=1$, and use the update rule
$$
  p^{k+1} = \frac{1-\zeta}{n} {\bf 1} + \zeta M p^{k}
$$
with $\zeta=0.85$ until the update amount is sufficiently small. For example you could keep iterating until $k=K$ when the following convergence is achieved.
$$\|p^{K+1}-p^{K}\|_2<10^{-6}$$

> a) How many iterations dis you need to get the convergence?
>
>
> b) Which webpage has the highest PageRank? List the PageRank of each page.

In [35]:
p=np.random.rand(n)   #위에 코드 실행한 후에 실행해야 오류 X
p/=np.linalg.norm(p,1) #양수이면서 p0=1만족
z=0.85

cnt=0

while True : 
  cnt+=1

  p1=p
  p2=(1-z)/n + z*M@p
  p=p2

  if np.linalg.norm(p2-p1) < 10**-6:
    print("Convergence in ",cnt)
    print(p)
    print("Page Rank 최댓값은",round(np.max(p),2),"이고 ",np.argmax(p)+1,"번째 페이지이다.")  
    break





Convergence in  80
[0.03278149 0.38440066 0.34291057 0.03908709 0.08088569 0.03908709
 0.01616948 0.01616948 0.01616948 0.01616948 0.01616948]
Page Rank 최댓값은 0.38 이고  2 번째 페이지이다.


---
_<div style="text-align: right"> 
Contents partially taken from Wikipedia (https://en.wikipedia.org/wiki/PageRank).
</div>_




b=0.03?